In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.3 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pickle
import sacrebleu
import evaluate
#from comet import download_model, load_from_checkpoint
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Load the dictionaries
dictionaries_direct = {
    "NLLB-200": {
        "finnish": "/content/drive/MyDrive/research_methods/results/translations/he_fi_dict_nllb_200_direct.pkl",
        "ukrainian": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_nllb_200_direct.pkl"
    },
    "Helsinki": {
        "finnish": "/content/drive/MyDrive/research_methods/results/translations/he_fi_dict_helsinki_direct.pkl",
        "ukrainian": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_helsinki_direct.pkl"
    },
    "Google": {
        "finnish": "/content/drive/MyDrive/research_methods/results/translations/he_fi_dict_google_direct.pkl",
        "ukrainian": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_google_direct.pkl"
    },
    "Llama": {
        "finnish": "/content/drive/MyDrive/research_methods/results/translations/he_fi_dict_llama3_direct.pkl",
        "ukrainian": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_llama3_direct.pkl"
    }
}

In [ ]:

# base translation dict
with open("/content/drive/Shareddrives/reaserch_methods/עותק של final_he_fi_dict.pkl", "rb") as f:
    he_fi_reference = pickle.load(f)

with open("/content/drive/Shareddrives/reaserch_methods/עותק של final_he_uk_dict.pkl", "rb") as f:
    he_uk_reference = pickle.load(f)



In [ ]:
model_path = download_model("Unbabel/wmt22-comet-da")
model = load_from_checkpoint(model_path)


In [ ]:
def calculate_metrics(translations_dict, reference_dict):
    """
    Calculate BLEU, METEOR, and COMET scores for translations
    """
    hypotheses = []
    references = []

    # Extract translations and references
    for key in translations_dict:
        if key in reference_dict:

            hyp = translations_dict[key][1]  #  model translation

            ref = reference_dict[key][1]     # base translation

            ##create list of base and model translations
            hypotheses.append(hyp)
            references.append([ref])

    # Calculate BLEU score
    bleu = sacrebleu.corpus_bleu(hypotheses, references) ## calculate on all sentences

    # Calculate METEOR score
    meteor = evaluate.load('meteor')
    meteor_score = meteor.compute(predictions=hypotheses, references=[ref[0] for ref in references])

    # Calculate COMET score
    model_path = download_model("Unbabel/wmt22-comet-da")
    model = load_from_checkpoint(model_path)

    # Prepare data for COMET
    comet_data = [
        {
            "src": translations_dict[key][0],  # Source text
            "mt": translations_dict[key][1],   # model translation
            "ref": reference_dict[key][1]      # base translation
        }
        for key in translations_dict if key in reference_dict
    ]

    comet_scores = model.predict(comet_data, batch_size=8, gpus=1)
    # Extract the system score from COMET output
    comet_score = float(comet_scores.system_score)

    return {
        "BLEU": bleu.score,
        "METEOR": meteor_score['meteor'],
        "COMET": comet_score
    }

In [ ]:
# Calculate metrics for each system and language pair
results = {}

for system_name, paths in dictionaries_direct.items():
    results[system_name] = {}

    # Finnish translations
    with open(paths["finnish"], "rb") as f:
        finnish_translations = pickle.load(f)
    results[system_name]["finnish"] = calculate_metrics(finnish_translations, he_fi_reference)

    # Ukrainian translations
    with open(paths["ukrainian"], "rb") as f:
        ukrainian_translations = pickle.load(f)
    results[system_name]["ukrainian"] = calculate_metrics(ukrainian_translations, he_uk_reference)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[n

KeyboardInterrupt: 

In [ ]:
for system_name, scores in results.items():
    print(f"\n{system_name} Results:")
    print("Finnish:")
    for metric, score in scores["finnish"].items():
        print(f"  {metric}: {score:.4f}")
    print("Ukrainian:")
    for metric, score in scores["ukrainian"].items():
        print(f"  {metric}: {score:.4f}")


NLLB-200 Results:
Finnish:
  BLEU: 5.9774
  METEOR: 0.3336
  COMET: 0.7841
Ukrainian:
  BLEU: 12.2693
  METEOR: 0.3658
  COMET: 0.7754

Helsinki Results:
Finnish:
  BLEU: 10.1798
  METEOR: 0.3561
  COMET: 0.8183
Ukrainian:
  BLEU: 15.3091
  METEOR: 0.3850
  COMET: 0.7650

Google Results:
Finnish:
  BLEU: 8.6004
  METEOR: 0.4141
  COMET: 0.8537
Ukrainian:
  BLEU: 24.3621
  METEOR: 0.4164
  COMET: 0.8400

Llama Results:
Finnish:
  BLEU: 5.4674
  METEOR: 0.2903
  COMET: 0.7664
Ukrainian:
  BLEU: 10.6909
  METEOR: 0.3216
  COMET: 0.7776


In [ ]:

# Load the dictionaries
dictionaries_pivot = {
    "NLLB-200": {
        "finnish": "/home/shirmash/research_methods/translations/he_fi_dict_nllb_200_pivot.pkl",
        "ukrainian": "/home/shirmash/research_methods/translations/he_uk_dict_nllb_200_pivot.pkl"
    },
    "Helsinki": {
        "finnish": "/home/shirmash/research_methods/translations/he_fi_dict_helsinki_pivot.pkl",
        "ukrainian": "/home/shirmash/research_methods/translations/he_uk_dict_helsinki_pivot.pkl"
    },
    "Google": {
        "finnish": "/home/shirmash/research_methods/translations/he_fi_dict_google_pivot.pkl",
        "ukrainian": "/home/shirmash/research_methods/translations/he_uk_dict_google_pivot.pkl"
    },
    "Llama": {
        "finnish": "/home/shirmash/research_methods/translations/he_fi_dict_llama3_pivot.pkl",
        "ukrainian": "/home/shirmash/research_methods/translations/he_uk_dict_llama3_pivot.pkl"
    }
}

In [ ]:
# Calculate metrics for each system and language pair
results = {}

for system_name, paths in dictionaries_pivot.items():
    results[system_name] = {}

    # Finnish translations
    with open(paths["finnish"], "rb") as f:
        finnish_translations = pickle.load(f)
    results[system_name]["finnish"] = calculate_metrics(finnish_translations, he_fi_reference)

    # Ukrainian translations
    with open(paths["ukrainian"], "rb") as f:
        ukrainian_translations = pickle.load(f)
    results[system_name]["ukrainian"] = calculate_metrics(ukrainian_translations, he_uk_reference)

# Print results
for system_name, scores in results.items():
    print(f"\n{system_name} Results:")
    print("Finnish:")
    for metric, score in scores["finnish"].items():
        print(f"  {metric}: {score:.4f}")
    print("Ukrainian:")
    for metric, score in scores["ukrainian"].items():
        print(f"  {metric}: {score:.4f}")

# ***calculate t-test***

In [ ]:

# base translation dict
with open("/content/drive/MyDrive/research_methods/results/final_he_fi_dict.pkl", "rb") as f:
    he_fi_reference = pickle.load(f)

with open("/content/drive/MyDrive/research_methods/results/final_he_uk_dict.pkl", "rb") as f:
    he_uk_reference = pickle.load(f)



In [ ]:
from scipy.stats import ttest_rel

In [ ]:

# Load the dictionaries
dictionaries = {
    # "NLLB-200": {
    #     "Uk_pivot": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_nllb_200_pivot.pkl",
    #     "Uk_direct": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_nllb_200_direct.pkl",
    # },
    # "Helsinki": {
    #     "Uk_pivot": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_helsinki_pivot.pkl",
    #     "Uk_direct": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_helsinki_direct.pkl",
    # },
    "Google": {
        "Uk_pivot": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_google_pivot.pkl",
        "Uk_direct": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_google_direct.pkl",
    },
    # "Llama": {
    #     "Uk_pivot": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_llama3_pivot.pkl",
    #     "Uk_direct": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_llama3_direct.pkl",
    # }
}

In [ ]:
meteor = evaluate.load('meteor')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# BLEU!!!
def calculate_t_test(translations_direct, translation_pivot, reference_dict):
    """
    Calculate BLEU, METEOR, and COMET scores for translations
    """
    hypotheses_direct = []
    hypotheses_pivot = []
    references = []

    # Extract translations and references
    for key in translations_direct:
        if key in reference_dict:

            hyp_direct = translations_direct[key][1]  #  model translation
            hyp_pivot = translation_pivot[key][1]
            ref = reference_dict[key][1]     # base translation

            ##create list of base and model translations
            hypotheses_direct.append(hyp_direct)
            hypotheses_pivot.append(hyp_pivot)
            references.append([ref])

    # Calculate BLEU score
    print(sacrebleu.corpus_bleu(hypotheses_direct, references)) ## calculate on all sentences
    print(sacrebleu.corpus_bleu(hypotheses_pivot, references)) ## calculate on all sentences
    # Calculate sentence-level BLEU scores
    direct_bleus = [sacrebleu.sentence_bleu(hyp, refs).score for hyp, refs in zip(hypotheses_direct, references)]
    pivot_bleus = [sacrebleu.sentence_bleu(hyp, refs).score for hyp, refs in zip(hypotheses_pivot, references)]
    t_stat, p_value = ttest_rel(direct_bleus, pivot_bleus)

    print(f"Direct BLEU scores: {direct_bleus}")
    print(f"Pivot BLEU scores: {pivot_bleus}")
    print(f"T-statistic: {t_stat}")
    print(f"P-value: {p_value}")

    # Interpretation
    if p_value < 0.05:
        significate = 'significate'
        print("The difference between Direct and Pivot BLEU scores is statistically significant.")
    else:
        print("No significant difference between Direct and Pivot BLEU scores.")
        significate = 'No significant'


    return {
        "T-statistic": t_stat,
        "P-value": p_value,
        "significate":significate
    }

In [ ]:
results = {}
for system_name, paths in dictionaries.items():
    results[system_name] = {}

    # Finnish translations
    with open(paths["Uk_direct"], "rb") as f:
        uk_translations_direct = pickle.load(f)
    with open(paths["Uk_pivot"], "rb") as f:
        uk_translations_pivot = pickle.load(f)
    results[system_name]["Ukrainian"] = calculate_t_test(uk_translations_direct, uk_translations_pivot, he_uk_reference)

results

BLEU = 11.50 81.6/27.0/5.6/1.4 (BP = 1.000 ratio = 1.000 hyp_len = 38 ref_len = 38)
BLEU = 7.79 78.6/14.6/2.5/1.3 (BP = 1.000 ratio = 1.000 hyp_len = 42 ref_len = 42)
Direct BLEU scores: 41.62799023755188
Pivot BLEU scores: 12.953235551535677
T-statistic: -9.74624154073323
P-value: 2.147098513806213e-22
The difference between Direct and Pivot BLEU scores is statistically significant.


{'Google': {'Ukrainian': {'T-statistic': -9.74624154073323,
   'P-value': 2.147098513806213e-22,
   'significate': 'significate'}}}

In [ ]:
# METEOR!!!!!!!!!!!!!!!!!!!!!!!!!
def calculate_t_test(translations_direct, translation_pivot, reference_dict):
    """
    Calculate BLEU, METEOR, and COMET scores for translations
    """
    hypotheses_direct = []
    hypotheses_pivot = []
    references = []

    # Extract translations and references
    for key in translations_direct:
        if key in reference_dict:

            hyp_direct = translations_direct[key][1]  #  model translation
            hyp_pivot = translation_pivot[key][1]
            ref = reference_dict[key][1]     # base translation

            ##create list of base and model translations
            hypotheses_direct.append(hyp_direct)
            hypotheses_pivot.append(hyp_pivot)
            references.append([ref])

    direct_meteor_scores = [
    meteor.compute(predictions=[hyp], references=[refs])["meteor"]
    for hyp, refs in zip(hypotheses_direct, references)
    ]

    pivot_meteor_scores = [
    meteor.compute(predictions=[hyp], references=[refs])["meteor"]
    for hyp, refs in zip(hypotheses_pivot, references)
    ]

    t_stat, p_value = ttest_rel(direct_meteor_scores, pivot_meteor_scores)

    print(f"Direct METEOR scores: {direct_meteor_scores[46]}")
    print(f"Pivot METEOR scores: {pivot_meteor_scores[46]}")
    print(f"T-statistic: {t_stat}")
    print(f"P-value: {p_value}")

    # Interpretation
    if p_value < 0.05:
        significate = 'significate'
        print("The difference between Direct and Pivot METEOR scores is statistically significant.")
    else:
        print("No significant difference between Direct and Pivot METEOR scores.")
        significate = 'No significant'


    return {
        "T-statistic": t_stat,
        "P-value": p_value,
        "significate":significate
    }

In [ ]:
results = {}
for system_name, paths in dictionaries.items():
    results[system_name] = {}

    # Finnish translations
    with open(paths["Uk_direct"], "rb") as f:
        uk_translations_direct = pickle.load(f)
    with open(paths["Uk_pivot"], "rb") as f:
        uk_translations_pivot = pickle.load(f)
    results[system_name]["Ukrainian"] = calculate_t_test(uk_translations_direct, uk_translations_pivot, he_fi_reference)
results

Direct METEOR scores: 0.5327181208053692
Pivot METEOR scores: 0.4619272516364239
T-statistic: -9.338762324431388
P-value: 1.073913882037986e-20
The difference between Direct and Pivot METEOR scores is statistically significant.


{'Google': {'Ukrainian': {'T-statistic': -9.338762324431388,
   'P-value': 1.073913882037986e-20,
   'significate': 'significate'}}}

In [ ]:

# COMET!!!!!!!!!!!!!!!!!
def calculate_t_test(translations_direct, translation_pivot, reference_dict):
    # Prepare data for COMET
    model_path = download_model("Unbabel/wmt22-comet-da")
    model = load_from_checkpoint(model_path)

    # Prepare data for COMET
    comet_data_direct = [
        {
            "src": translations_direct[key][0],  # Source text
            "mt": translations_direct[key][1],   # model translation
            "ref": reference_dict[key][1]      # base translation
        }
        for key in translations_direct if key in reference_dict
    ]

    comet_data_pivot = [
        {
            "src": translation_pivot[key][0],  # Source text
            "mt": translation_pivot[key][1],   # model translation
            "ref": reference_dict[key][1]      # base translation
        }
        for key in translation_pivot if key in reference_dict
    ]

    direct_scores = model.predict(comet_data_direct, batch_size=8, gpus=1)["scores"]
    pivot_scores = model.predict(comet_data_pivot, batch_size=8, gpus=1)["scores"]

    t_stat, p_value = ttest_rel(direct_scores, pivot_scores)

    print(f"Direct COMET scores: {direct_scores}")
    print(f"Pivot COMET scores: {pivot_scores}")
    print(f"T-statistic: {t_stat}")
    print(f"P-value: {p_value}")

    # Interpretation
    if p_value < 0.05:
        significate = 'significate'
        print("The difference between Direct and Pivot COMET scores is statistically significant.")
    else:
        print("No significant difference between Direct and Pivot COMET scores.")
        significate = 'No significant'


    return {
        "T-statistic": t_stat,
        "P-value": p_value,
        "significate":significate
    }


In [ ]:
results = {}
for system_name, paths in dictionaries.items():
    results[system_name] = {}

    # Finnish translations
    with open(paths["Uk_direct"], "rb") as f:
        uk_translations_direct = pickle.load(f)
    with open(paths["Uk_pivot"], "rb") as f:
        uk_translations_pivot = pickle.load(f)
    results[system_name]["Ukrainian"] = calculate_t_test(uk_translations_direct, uk_translations_pivot, he_uk_reference)
results

# print sentences

In [ ]:
import pickle

In [ ]:

# Load the dictionaries
dictionaries_direct = {
    "Google": {
        "uk_pivot": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_google_pivot.pkl",
        "uk_direct": "/content/drive/MyDrive/research_methods/results/translations/he_uk_dict_google_direct.pkl",
    }
}

In [ ]:
def print_sentence_example(translations_direct, translation_pivot, reference_dict):
    """
    Calculate BLEU, METEOR, and COMET scores for translations
    """
    hypotheses_direct = []
    hypotheses_pivot = []
    references = []
    count = 0
    # Extract translations and references
    for key in translations_direct:
        if key in reference_dict:

            hyp_direct = translations_direct[key][1]  #  model translation
            hyp_pivot = translation_pivot[key][1]
            ref = reference_dict[key][1]     # base translation
            if count == 46:
                print(reference_dict[key])
                print(translations_direct[key])
                print(translation_pivot[key])
                break
            count+=1



In [ ]:
results = {}
for system_name, paths in dictionaries_direct.items():
    results[system_name] = {}

    # Finnish translations
    with open(paths["uk_direct"], "rb") as f:
        uk_translations_direct = pickle.load(f)
    with open(paths["uk_pivot"], "rb") as f:
        uk_translations_pivot = pickle.load(f)
    results[system_name]["uk"] = print_sentence_example(uk_translations_direct, uk_translations_pivot, he_uk_reference)


["אני רוצה שתדעו שאני , אחי ואחותי באמת אוהבים לאכול צ ' יפס כרוב אפוי ", 'Haluan teidän tietävän , että minä , veljeni ja sisareni itse asiassa pidämme paistetuista kaalilastuista ']
["אני רוצה שתדעו שאני , אחי ואחותי באמת אוהבים לאכול צ ' יפס כרוב אפוי ", 'Haluan sinun tietävän, että minä, veljeni ja siskoni syömme todella mielellään uunikaalialastuja ']
["אני רוצה שתדעו שאני , אחי ואחותי באמת אוהבים לאכול צ ' יפס כרוב אפוי ", 'Haluan sinun tietävän, että veljeni, sisareni ja minä todella tykkäämme syödä uunikaalialastuja ']
